In [ ]:
import tensorflow as tf

BATCH_SIZE = 16
SEED = 1

# Training set
train_generator = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-ecg-images/train',
    batch_size=BATCH_SIZE,
    image_size=(512, 1024),
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    label_mode='categorical',
    subset='training',
    seed=SEED
)

# Validation set
val_generator = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-ecg-images/train',
    batch_size=BATCH_SIZE,
    image_size=(512, 1024),
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    label_mode='categorical',
    subset='validation',
    seed=SEED
)

In [ ]:
from tensorflow.keras.applications import EfficientNetV2S

input_shape = (512,1024,3)

base_model = EfficientNetV2S(weights='imagenet', 
                      include_top=False,
                      input_shape=input_shape)

In [ ]:
for layer in base_model.layers: # freezes all the layers in the base model
  layer.trainable = False

# base_model.summary()

In [ ]:
iterator = iter(train_generator)

In [ ]:
import numpy as np

next(iterator)[0][0].numpy().shape

In [ ]:
import matplotlib.pyplot as plt # library that allows us to visualize the data


plt.imshow(next(iterator)[0][0].numpy().astype(np.int32)) # shows each image

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

def create_model():
    return Sequential([
        Input(batch_size = 16, shape=input_shape),
        base_model,  # Pre-trained base model
        GlobalAveragePooling2D(),  # Global average pooling layer
        Dense(1024, activation='relu'),  # Custom dense layer for learning
        Dropout(0.2),  # Dropout for regularization
        Dense(512, activation='relu'),  # Another custom dense layer
        Dropout(0.2),  # Another dropout layer
        Dense(4, activation='softmax')  # multiclass classfication
    ])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# stops the training early if there is no improvement
early_stopping = EarlyStopping(
    patience = 5,
    monitor = 'val_loss',
    restore_best_weights = True,
    verbose = 1
)

In [ ]:
from tensorflow.keras.metrics import *

metrics = ['accuracy', Precision(), Recall(), AUC(),'f1_score']

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

model = create_model()
model.compile(loss=CategoricalCrossentropy(),
                optimizer=Adam(learning_rate=0.0001),
                metrics=metrics)
model.summary()

In [ ]:
# trains the neural network
history = model.fit(train_generator,
            epochs = 100,
            validation_data = val_generator,
            callbacks = [early_stopping])

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
def plot_accuracy(history):
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

# Call the function with your model's history object
plot_accuracy(history)

In [ ]:
import pickle

# Save history to a file
with open('history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# save model
model.save('ecg_classifier_model.h5')
!zip -r ecg_classifier_model.zip ecg_classifier_model.h5

In [ ]:
import numpy as np

test_dataset = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/new-ecg-images/test',
    image_size=(512, 1024),
    label_mode = 'categorical')

test_dataset = test_dataset.unbatch()

# Look at first few batches
ecg_test_images = []
test_labels = []
for images, labels in test_dataset:
    ecg_test_images.append(images)
    test_labels.append(labels)  # Convert tensor to numpy array

images_list = np.array(ecg_test_images)
labels_list = np.array(test_labels)

In [ ]:
test_loss, test_accuracy, test_precision, test_recall, test_auc, test_f1 = model.evaluate(images_list, labels_list)
print(f'Accuracy is {test_accuracy:.2%}.')
print(f'Precision is {test_precision:.2%}.')
print(test_recall)
print(f"{2*test_precision*test_recall/(test_precision+test_recall):.2%}")

In [ ]:
print(test_loss)
print(test_auc)

In [ ]:
predictions = model.predict(images_list)
predicted_classes = np.argmax(predictions, axis=1)
# predicted_classes = to_categorical(predicted_classes)

print(predicted_classes[0])

class_map = {
    0: 'Myocardial Infarction (MI)',
    1: 'History of MI',
    2: 'Abnormal Heartbeat',
    3: 'Normal Heartbeat'
}

labeled_predictions = [class_map[pred] for pred in predicted_classes]

print(f'Prediction: {labeled_predictions[0]}')

In [ ]:
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical

y_true = labels_list
y_pred = to_categorical(predicted_classes)
target_names = ["MI", "History of MI", "Abnormal", "Normal"]

print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

In [ ]:
# Convert one-hot encoded labels to class indices
y_true_classes = np.argmax(y_true, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)
plt.show()